In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
path = '/content/gdrive/My Drive/Outsourcing/DACON/MVtec/'

import os
os.chdir(path)

!pip install timm
!pip install torchsampler

In [ ]:

import torch.optim as optim
import albumentations as A
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import random
import torch
import timm
import cv2
import os
import time
import sys

from torchsampler.imbalanced import ImbalancedDatasetSampler
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader
from glob import glob
from tqdm import tqdm
from sklearn.metrics import f1_score

device = torch.device('cuda')

In [ ]:
def img_load(path, resize=512):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (resize, resize))
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return img


In [ ]:
train_png = sorted(glob('open/train/*.png'))

train_y = pd.read_csv("open/train_df.csv")
train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
# 주어진 레이블마다 인덱스를 부여
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]
train_imgs = [img_load(m) for m in tqdm(train_png)]

In [ ]:
print("사물의 상태: ", np.unique(train_y["state"]))
print("사물의 상태 개수: ", len(np.unique(train_y["state"])))

print("-" * 80)

print("클래스: ", np.unique(train_y["class"]))
print("클래스 개수: ", len(np.unique(train_y["class"])))

print("-" * 80)

print("레이블: ", np.unique(train_y["label"]))
print("레이블 개수: ", len(np.unique(train_y["label"])))

사물의 상태:  ['bent' 'bent_lead' 'bent_wire' 'broken' 'broken_large' 'broken_small'
 'broken_teeth' 'cable_swap' 'color' 'combined' 'contamination' 'crack'
 'cut' 'cut_inner_insulation' 'cut_lead' 'cut_outer_insulation'
 'damaged_case' 'defective' 'fabric_border' 'fabric_interior'
 'faulty_imprint' 'flip' 'fold' 'glue' 'glue_strip' 'good' 'gray_stroke'
 'hole' 'liquid' 'manipulated_front' 'metal_contamination' 'misplaced'
 'missing_cable' 'missing_wire' 'oil' 'pill_type' 'poke' 'poke_insulation'
 'print' 'rough' 'scratch' 'scratch_head' 'scratch_neck' 'split_teeth'
 'squeeze' 'squeezed_teeth' 'thread' 'thread_side' 'thread_top']
사물의 상태 개수:  49
--------------------------------------------------------------------------------
클래스:  ['bottle' 'cable' 'capsule' 'carpet' 'grid' 'hazelnut' 'leather'
 'metal_nut' 'pill' 'screw' 'tile' 'toothbrush' 'transistor' 'wood'
 'zipper']
클래스 개수:  15
--------------------------------------------------------------------------------
레이블:  ['bottle-broken_large'

In [ ]:
anomaly_df = train_y[train_y["state"] != "good"]
normal_df = train_y[train_y["state"] == "good"]

print("전체 학습 데이터 개수: ", len(train_y))
print("이상치: ", len(anomaly_df), "| 정상치: ", len(normal_df), f"| 이상치-정상치 비율: 1:{len(normal_df)/len(anomaly_df)}")

전체 학습 데이터 개수:  4277
이상치:  648 | 정상치:  3629 | 이상치-정상치 비율: 1:5.6003086419753085


In [ ]:
total_number_per_label = {}
for label in np.unique(train_y["label"]):
    num_data_per_label = len(train_y[train_y['label'] == label])
    total_number_per_label[label] = num_data_per_label


total_number_per_label = sorted(total_number_per_label.items(), key=lambda x: x[1], reverse=True)
total_number_per_label = dict(total_number_per_label)

print(f"{'Class-State':<35} | {'Count':>30}")
print('-' * 80)
for idx, (key, value) in enumerate(total_number_per_label.items()):
    print(f"{idx+1:03d}. {key:<30} | {value:>30}")

Class-State                         |                          Count
--------------------------------------------------------------------------------
001. hazelnut-good                  |                            391
002. screw-good                     |                            320
003. carpet-good                    |                            280
004. pill-good                      |                            267
005. grid-good                      |                            264
006. wood-good                      |                            247
007. leather-good                   |                            245
008. zipper-good                    |                            240
009. tile-good                      |                            230
010. cable-good                     |                            224
011. metal_nut-good                 |                            220
012. capsule-good                   |                            219
013. transistor-good  

# 생각해볼 수 있는 것들
1. 정상이 없는 class가 있을 수 있다.
    - 정상이 없는 class가 있는가? 있다면 무엇인가?

2. class 마다 모든 state가 존재하지 않을 수 있다.
    - class 마다 존재하는 state와 그 개수를 를 정리해보자.
    - 즉, 모든 클래스마다 49가지의 상태를 갖고 있지 않다.

---
# 풀고자하는 문제
* 불균형 데이터 셋을 학습하여 사물의 상태를 잘 분류할 수 있는 알고리즘 만들기


# 접근 방법
## Supervised Learning
1. 클래스 imbalance를 해결해서 supervised learning 방법론으로 classification 문제로 접근.

2. state 별로 데이터를 묶고, state 별로 따로 모델을 만들어 학습 한 다음 앙상블을 한다. (49개의 모델이 나옴)

3. class 별로 모델을 만들어 학습 한 다음 앙상블을 한다.
(15개의 모델이 나옴)

## Unsupervised Learning
* 'MVTec AD— A Comprehensive Real-World Dataset for Unsupervised Anomaly Detection'
논문에서 방법론 소개
    - 불균형의 정의를 고장 데이터 간의 개수 불균형으로 정의 한다면 이 접근 방법으로 가능할 듯.